Problem 2: Ride-Sharing Route Optimization
A ride-sharing app needs to pick up 3 passengers. Design an algorithm to find the best pickup order.

Define the state space (driver location + which passengers are picked up)
Define actions (go to passenger X, go to destination)
Use A* with a heuristic based on distances
What heuristic would you use? Is it admissible?

In [1]:
print("let's start with the question 2")

let's start with the question 2


In [ ]:
import heapq
import math


# Helper: Euclidean distance
# ----------------------------
def distance(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)



# A* Algorithm
# ----------------------------
def astar_pickup(start, passengers):
    """
    start: (x, y)
    passengers: dict { 'P1': (x, y), 'P2': (x, y), 'P3': (x, y) }
    """

    # State = (current_location, frozenset_of_picked_passengers)
    initial_state = (start, frozenset())

    # Priority queue: (f, g, state, path)
    pq = []
    heapq.heappush(pq, (0, 0, initial_state, []))

    visited = {}

    while pq:
        f, g, state, path = heapq.heappop(pq)
        current_loc, picked = state

        # Goal check
        if len(picked) == len(passengers):
            return path, g

        # Avoid reprocessing worse states
        if state in visited and visited[state] <= g:
            continue
        visited[state] = g

        # Expand to all unpicked passengers
        for p_name, p_loc in passengers.items():
            if p_name not in picked:
                new_picked = frozenset(set(picked) | {p_name})
                cost = distance(current_loc, p_loc)
                new_g = g + cost

                # Heuristic: nearest unpicked passenger
                remaining = [
                    distance(p_loc, passengers[r])
                    for r in passengers
                    if r not in new_picked
                ]
                h = min(remaining) if remaining else 0

                new_f = new_g + h

                new_state = (p_loc, new_picked)
                new_path = path + [p_name]

                heapq.heappush(pq, (new_f, new_g, new_state, new_path))

    return None, float('inf')


# Example Usage
# ----------------------------
if __name__ == "__main__":

    start_location = (0, 0)

    passengers = {
        "P1": (2, 3),
        "P2": (5, 1),
        "P3": (6, 4)
    }

    best_order, total_cost = astar_pickup(start_location, passengers)

    print("Best pickup order:", best_order)
    print("Total distance:", round(total_cost, 2))


Best pickup order: ['P1', 'P2', 'P3']
Total distance: 10.37


In [5]:
"""
=============================================================================
A* SEARCH ALGORITHM - Complete Implementation
=============================================================================
Author: For FAST NUCES AI Lab
Purpose: Find the optimal path using heuristic guidance

THE A* FORMULA:
    f(n) = g(n) + h(n)
    
    f(n) = Total estimated cost (used to decide which node to expand)
    g(n) = Actual cost from start to node n
    h(n) = Estimated cost from node n to goal (heuristic)

KEY INSIGHT:
- If h(n) = 0, A* becomes UCS (no heuristic guidance)
- If g(n) = 0, A* becomes Greedy Best-First (ignores actual cost)
- A* balances both for optimal performance!

ADMISSIBILITY:
- Heuristic must NEVER overestimate the true cost
- If h(n) says "5 steps left" but it's actually 8, that's OK (underestimate)
- If h(n) says "10 steps left" but it's actually 6, that's BAD (overestimate)

REAL WORLD USES:
- Video game pathfinding
- GPS navigation
- Robot motion planning
- Network routing
=============================================================================
"""

import heapq
import math

def manhattan_distance(pos1, pos2):
    """Manhattan distance: |x1-x2| + |y1-y2|"""
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])


def euclidean_distance(pos1, pos2):
    """Euclidean distance: straight line"""
    return math.sqrt((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)


def astar(grid, start, goal, heuristic=manhattan_distance):
    """
    A* Search on a 2D grid.
    
    Parameters:
        grid: 2D list where 0=walkable, 1=wall
        start: (row, col) starting position
        goal: (row, col) target position
        heuristic: Function to estimate distance to goal
    
    Returns:
        Path as list of positions, or None if no path
    """
    
    rows, cols = len(grid), len(grid[0])
    
    # Priority queue: (f_score, g_score, position, path)
    pq = []
    start_h = heuristic(start, goal)
    heapq.heappush(pq, (start_h, 0, start, [start]))
    
    # Best g_score to reach each position
    g_scores = {start: 0}
    
    # Directions: up, down, left, right
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    
    while pq:
        f, g, current, path = heapq.heappop(pq)
        
        # Goal check
        if current == goal:
            return path
        
        # Skip if we found a better path already
        if g > g_scores.get(current, float('inf')):
            continue
        
        # Explore neighbors
        for dr, dc in directions:
            nr, nc = current[0] + dr, current[1] + dc
            neighbor = (nr, nc)
            
            # Check bounds and walls
            if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == 0:
                new_g = g + 1
                new_h = heuristic(neighbor, goal)
                new_f = new_g + new_h  # THE A* FORMULA!
                
                if new_g < g_scores.get(neighbor, float('inf')):
                    g_scores[neighbor] = new_g
                    heapq.heappush(pq, (new_f, new_g, neighbor, path + [neighbor]))
    
    return None


# ==================== TEST THE CODE ====================
if __name__ == "__main__":
    
    # Create a grid (0=open, 1=wall)
    grid = [
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]
    ]
    
    start = (0, 0)
    goal = (5, 7)
    
    path = astar(grid, start, goal)
    
    if path:
        print("✓ Path found!")
        print("  Length:", len(path), "steps")
        
        # Visualize
        print("\n  Grid (S=start, G=goal, *=path, #=wall):")
        path_set = set(path)
        for r in range(len(grid)):
            row_str = "  "
            for c in range(len(grid[0])):
                if (r,c) == start: row_str += "S "
                elif (r,c) == goal: row_str += "G "
                elif (r,c) in path_set: row_str += "* "
                elif grid[r][c] == 1: row_str += "# "
                else: row_str += ". "
            print(row_str)

✓ Path found!
  Length: 13 steps

  Grid (S=start, G=goal, *=path, #=wall):
  S * * * * * * * 
  . . . # # . . * 
  . . . # # . . * 
  . . . # # . . * 
  . . . . . . . * 
  . . . . . . . G 
